In [1]:
import xarray as xr 
import numpy as np

import sys
sys.path.append('/home/users/cturrell/documents/eddy_feedback')
# pylint: disable=wrong-import-position
import functions.data_wrangling as data

### CanESM5

epfy (300)
- EmonZ: 39 plev

ua (300)
- Amon:
- AERmon:

In [2]:
can_epfy = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/epfy/CanESM5/*.nc', 
                         combine='nested', concat_dim='ens_ax', parallel=True)
can_epfy 

<xarray.Dataset>
Dimensions:    (time: 14, ens_ax: 300, bnds: 2, plev: 39, lat: 64)
Coordinates:
  * time       (time) object 2000-04-16 00:00:00 ... 2001-05-16 12:00:00
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 10.0 7.0 5.0 3.0
  * lat        (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
Dimensions without coordinates: ens_ax, bnds
Data variables:
    time_bnds  (ens_ax, time, bnds) object dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    lat_bnds   (ens_ax, lat, bnds) float64 dask.array<chunksize=(1, 64, 2), meta=np.ndarray>
    epfy       (ens_ax, time, plev, lat) float32 dask.array<chunksize=(1, 1, 39, 64), meta=np.ndarray>
Attributes: (12/52)
    CCCma_model_hash:            37a65f96056a7aa4fa354897986cc70301801f67
    CCCma_parent_runid:          p2-amip-ru10
    CCCma_pycmor_hash:           8157ca33437ab096d583668ba0da0c2ce55acfe1
    CCCma_runid:                 pamip11-100
    Conventions:                 CF-1.7 CMIP-6.2
    YMDH_branch_time_in_child:   2000:04:01:00
    ...                          ...
    tracking_id:                 hdl:21.14100/f30700de-4b1e-47b8-b50a-82c5595...
    variable_id:                 epfy
    variant_label:               r100i1p2f1
    version:                     v20190429
    license:                     CMIP6 model data produced by The Government ...
    cmor_version:                3.4.0

In [3]:
can_ua = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/ua/CanESM5/Amon/*.nc', 
                         combine='nested', concat_dim='ens_ax', parallel=True)
can_ua

HDF5-DIAG: Error detected in HDF5 (1.14.2) thread 1:
  #000: H5F.c line 836 in H5Fopen(): unable to synchronously open file
    major: File accessibility
    minor: Unable to open file
  #001: H5F.c line 796 in H5F__open_api_common(): unable to open file
    major: File accessibility
    minor: Unable to open file
  #002: H5VLcallback.c line 3863 in H5VL_file_open(): open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLcallback.c line 3675 in H5VL__file_open(): open failed
    major: Virtual Object Layer
    minor: Can't open object
  #004: H5VLnative_file.c line 128 in H5VL__native_file_open(): unable to open file
    major: File accessibility
    minor: Unable to open file
  #005: H5Fint.c line 2003 in H5F_open(): unable to read superblock
    major: File accessibility
    minor: Read failed
  #006: H5Fsuper.c line 600 in H5F__super_read(): truncated file: eof = 163486, sblock->base_addr = 0, stored_eof = 8063443
    major: File accessibility
    minor

OSError: [Errno -101] NetCDF: HDF error: '/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/ua/CanESM5/Amon/ua_Amon_CanESM5_pdSST-pdSIC_r36i1p2f1_gn_200004-200105.nc'

In [4]:
can_ua = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/ua/CanESM5/AERmon/*.nc', 
                         combine='nested', concat_dim='ens_ax', parallel=True)
can_ua

<xarray.Dataset>
Dimensions:    (time: 14, ens_ax: 300, bnds: 2, plev: 39, lat: 64)
Coordinates:
  * time       (time) object 2000-04-16 00:00:00 ... 2001-05-16 12:00:00
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 10.0 7.0 5.0 3.0
  * lat        (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
Dimensions without coordinates: ens_ax, bnds
Data variables:
    time_bnds  (ens_ax, time, bnds) object dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    lat_bnds   (ens_ax, lat, bnds) float64 dask.array<chunksize=(1, 64, 2), meta=np.ndarray>
    ua         (ens_ax, time, plev, lat) float32 dask.array<chunksize=(1, 1, 39, 64), meta=np.ndarray>
Attributes: (12/52)
    CCCma_model_hash:            37a65f96056a7aa4fa354897986cc70301801f67
    CCCma_parent_runid:          p2-amip-ru10
    CCCma_pycmor_hash:           8157ca33437ab096d583668ba0da0c2ce55acfe1
    CCCma_runid:                 pamip11-100
    Conventions:                 CF-1.7 CMIP-6.2
    YMDH_branch_time_in_child:   2000:04:01:00
    ...                          ...
    tracking_id:                 hdl:21.14100/a3f4d3d5-b1bf-42f4-a679-ec7c434...
    variable_id:                 ua
    variant_label:               r100i1p2f1
    version:                     v20190429
    license:                     CMIP6 model data produced by The Government ...
    cmor_version:                3.4.0

### NorESM2-LM

epfy (200)
- EmonZ: 39 plev

ua (200)
- Emon: 9 plev
- Amon:
- AERmon:

In [5]:
nor_epfy = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/epfy/NorESM2-LM/*.nc', 
                         combine='nested', concat_dim='ens_ax', parallel=True)
nor_epfy 

<xarray.Dataset>
Dimensions:    (time: 12, ens_ax: 200, bnds: 2, plev: 39, lat: 96)
Coordinates:
  * time       (time) object 2000-06-16 00:00:00 ... 2001-05-16 12:00:00
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 10.0 7.0 5.0 3.0
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
Dimensions without coordinates: ens_ax, bnds
Data variables:
    time_bnds  (ens_ax, time, bnds) object dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    lat_bnds   (ens_ax, lat, bnds) float64 dask.array<chunksize=(1, 96, 2), meta=np.ndarray>
    epfy       (ens_ax, time, plev, lat) float32 dask.array<chunksize=(1, 1, 39, 96), meta=np.ndarray>
Attributes: (12/48)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               PAMIP
    branch_method:             Hybrid-restart from year 2000-04-01 of amip
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     9215.0
    ...                        ...
    title:                     NorESM2-LM output prepared for CMIP6
    variable_id:               epfy
    variant_label:             r100i1p1f1
    license:                   CMIP6 model data produced by NCC is licensed u...
    cmor_version:              3.5.0
    tracking_id:               hdl:21.14100/43baa687-cf69-4640-b923-613139c0e245

In [6]:
nor_ua = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/ua/NorESM2-LM/Emon/*.nc', 
                         combine='nested', concat_dim='ens_ax', parallel=True)
nor_ua

<xarray.Dataset>
Dimensions:    (time: 12, ens_ax: 200, bnds: 2, plev: 7, lat: 96, lon: 144)
Coordinates:
  * time       (time) object 2000-06-16 00:00:00 ... 2001-05-16 12:00:00
  * plev       (plev) float64 9.25e+04 8.5e+04 7e+04 6e+04 5e+04 2.5e+04 5e+03
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Dimensions without coordinates: ens_ax, bnds
Data variables:
    time_bnds  (ens_ax, time, bnds) object dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    lat_bnds   (ens_ax, lat, bnds) float64 dask.array<chunksize=(1, 96, 2), meta=np.ndarray>
    lon_bnds   (ens_ax, lon, bnds) float64 dask.array<chunksize=(1, 144, 2), meta=np.ndarray>
    ua         (ens_ax, time, plev, lat, lon) float32 dask.array<chunksize=(1, 1, 7, 96, 144), meta=np.ndarray>
Attributes: (12/49)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               PAMIP
    branch_method:             Hybrid-restart from year 2000-04-01 of amip
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     9215.0
    ...                        ...
    title:                     NorESM2-LM output prepared for CMIP6
    variable_id:               ua
    variant_label:             r100i1p1f1
    license:                   CMIP6 model data produced by NCC is licensed u...
    cmor_version:              3.5.0
    tracking_id:               hdl:21.14100/11eb0c0a-6c20-47d1-80b3-f5e23b68f0f8

In [7]:
nor_ua = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/ua/NorESM2-LM/Amon/*.nc', 
                         combine='nested', concat_dim='ens_ax', parallel=True)
nor_ua

<xarray.Dataset>
Dimensions:    (time: 12, ens_ax: 200, bnds: 2, plev: 19, lat: 96, lon: 144)
Coordinates:
  * time       (time) object 2000-06-16 00:00:00 ... 2001-05-16 12:00:00
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Dimensions without coordinates: ens_ax, bnds
Data variables:
    time_bnds  (ens_ax, time, bnds) object dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    lat_bnds   (ens_ax, lat, bnds) float64 dask.array<chunksize=(1, 96, 2), meta=np.ndarray>
    lon_bnds   (ens_ax, lon, bnds) float64 dask.array<chunksize=(1, 144, 2), meta=np.ndarray>
    ua         (ens_ax, time, plev, lat, lon) float32 dask.array<chunksize=(1, 1, 19, 96, 144), meta=np.ndarray>
Attributes: (12/49)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               PAMIP
    branch_method:             Hybrid-restart from year 2000-04-01 of amip
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     9215.0
    ...                        ...
    title:                     NorESM2-LM output prepared for CMIP6
    variable_id:               ua
    variant_label:             r100i1p1f1
    license:                   CMIP6 model data produced by NCC is licensed u...
    cmor_version:              3.5.0
    tracking_id:               hdl:21.14100/a2aa1624-e316-436b-be2e-e20e3db1549e

In [8]:
nor_ua = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/monthly/pdSST-pdSIC/ua/NorESM2-LM/AERmon/*.nc', 
                         combine='nested', concat_dim='ens_ax', parallel=True)
nor_ua

<xarray.Dataset>
Dimensions:    (time: 12, ens_ax: 200, bnds: 2, lev: 32, lat: 96, lon: 144)
Coordinates:
  * time       (time) object 2000-06-16 00:00:00 ... 2001-05-16 12:00:00
  * lev        (lev) float64 0.9926 0.9763 0.9575 ... 0.01436 0.007595 0.003643
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Dimensions without coordinates: ens_ax, bnds
Data variables:
    time_bnds  (ens_ax, time, bnds) object dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
    lev_bnds   (ens_ax, lev, bnds) float64 dask.array<chunksize=(1, 32, 2), meta=np.ndarray>
    p0         (ens_ax) float64 1e+05 1e+05 1e+05 1e+05 ... 1e+05 1e+05 1e+05
    a          (ens_ax, lev) float64 dask.array<chunksize=(1, 32), meta=np.ndarray>
    b          (ens_ax, lev) float64 dask.array<chunksize=(1, 32), meta=np.ndarray>
    ps         (ens_ax, time, lat, lon) float32 dask.array<chunksize=(1, 1, 96, 144), meta=np.ndarray>
    a_bnds     (ens_ax, lev, bnds) float64 dask.array<chunksize=(1, 32, 2), meta=np.ndarray>
    b_bnds     (ens_ax, lev, bnds) float64 dask.array<chunksize=(1, 32, 2), meta=np.ndarray>
    lat_bnds   (ens_ax, lat, bnds) float64 dask.array<chunksize=(1, 96, 2), meta=np.ndarray>
    lon_bnds   (ens_ax, lon, bnds) float64 dask.array<chunksize=(1, 144, 2), meta=np.ndarray>
    ua         (ens_ax, time, lev, lat, lon) float32 dask.array<chunksize=(1, 1, 32, 96, 144), meta=np.ndarray>
Attributes: (12/49)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               PAMIP
    branch_method:             Hybrid-restart from year 2000-04-01 of amip
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     9215.0
    ...                        ...
    title:                     NorESM2-LM output prepared for CMIP6
    variable_id:               ua
    variant_label:             r100i1p1f1
    license:                   CMIP6 model data produced by NCC is licensed u...
    cmor_version:              3.5.0
    tracking_id:               hdl:21.14100/1a36ad74-15a4-476d-ba43-de08cd53a24a